In [1]:
import grpc
from proto.message_log_pb2 import LogRequest
from proto.message_log_pb2 import NOTSET, DEBUG, INFO, WARNING, ERROR, CRITICAL
from proto.message_log_pb2_grpc import MessageLoggerStub

LOG_END_POINT = 'localhost:36000'


def request_log(request):
    with grpc.insecure_channel(LOG_END_POINT) as channel:
        stub = MessageLoggerStub(channel)
        response = stub.LogMessage(request)

        return response

In [2]:

from transformers import AutoTokenizer, AutoModelForCausalLM

# path를 변경해서 모델을 돌림
# "EleutherAI/polyglot-ko-5.8b"
# "./outputs/GT-1.3B-143-5"
# "./outputs/GT-1.3B-428-15w-1e-6/final_model"

# path = "EleutherAI/polyglot-ko-5.8b"
path = "./outputs/BigDemo-1.3B-541-5w-3"
tokenizer = AutoTokenizer.from_pretrained(path)
# Half precision
# if 1.3B no half()
model = AutoModelForCausalLM.from_pretrained(path).half()
# model = AutoModelForCausalLM.from_pretrained(path)

In [3]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# CPU로 하면 문제해결?
model.to(device)
print(device)

cuda


In [4]:
!nvidia-smi

Tue Jan 17 23:30:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.48.07    Driver Version: 515.48.07    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1D.0 Off |                    0 |
| N/A   50C    P0    35W /  70W |  10069MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
bad_words = [
  '...',
  '....',
  '(중략)',
  'http'
]

In [6]:
model.eval()
def predict(text):
    with torch.no_grad():
        input_ids = tokenizer.encode(text)
        tokens = torch.tensor([input_ids])
        tokens = tokens.to(device)
        # generate and end generate if <|endoftext|> is not in text
        # 길어질수록 살릴 후보가 많아야 할 것 같다 top_k, top_p 키우기
        gen_ids = model.generate(tokens,
                                 do_sample=True,
                                 max_new_tokens=128,
                                 top_k=100,
                                 top_p=0.9,
                                 temperature=0.7,
                                 repetition_penalty=1.2,
                                 num_return_sequences=3,
                                 pad_token_id=tokenizer.pad_token_id,
                                 eos_token_id=tokenizer.eos_token_id,
                                 bos_token_id=tokenizer.bos_token_id,
                                #  use_cache=True,
                                 )
        generated = tokenizer.batch_decode(gen_ids)
        return generated


In [17]:
# text = input("입력: ")
# 삶은 나를 싫어하네, 내가 가는 길마다 방해하니
# text = """나는 제목과 카테고리가 주어지면 알맞는 글을 작성하는 작가이다.
# 민주화 운동을 하다가 친구들이 다치고 잡혀간다. 
text = """
제목: 6월 민주 항쟁
카테고리: 20대 초반 민주화라는 담대한 목표를 가지고 친구들과 함께 민주화 운동을 하던 시절을 회상하는 내용 담은 수필
필자: 민주운동을 관심을 가지고 6월 민주 항쟁 시위를 하던 대학생 
내용: 민주화운동이 일어난 지 40년, 우리나라의 민주주의가 이만큼 성장한 건 분명 사실이나 여전히 사회 곳곳에서 많은 이들이 부당함과 억울함 속에 살아가야만 한다."""


result = predict(text)

for x in result:
    x = x.replace("<|endoftext|>", "")
    print(x)

if "outputs" in path[0:10]:
    model_name = path[10:]
else:
    model_name = path
msg = f'Model: {model_name}\nPrompt: {text}\n'
for x in result:
    x = x.replace("<|endoftext|>", "")
    msg += f'Generated: {x}\n'

log_request = LogRequest(log_level=DEBUG, msg=msg)

log_response = request_log(log_request)
assert log_response.done is True

# 보험
torch.cuda.empty_cache()


제목: 6월 민주 항쟁
카테고리: 20대 초반 민주화라는 담대한 목표를 가지고 친구들과 함께 민주화 운동을 하던 시절을 회상하는 내용 담은 수필
필자: 민주운동을 관심을 가지고 6월 민주 항쟁 시위를 하던 대학생 
내용: 민주화운동이 일어난 지 40년, 우리나라의 민주주의가 이만큼 성장한 건 분명 사실이나 여전히 사회 곳곳에서 많은 이들이 부당함과 억울함 속에 살아가야만 한다. 내 집 문 앞마당까지 들어온 도둑고양이처럼 아무 죄 없어도 죄인 취급 받으며 살아야 할 사람도 있다.....우리 모두에게 조금 더 공평하고 정의로운 세상이란 과연 올 수나있을까? 아직 한참 먼 것 같지만 포기할 수도 물러설수도 없이 계속 나아가기로 했으니까 끝끝내 버티어 나가 보려고 노력해야겠다....민주주의란 단어 자체엔 크게 의미두지 않았는데 지금 생각해보면 우리에겐 정말 소중했구나 싶기도 하고 참 감사히 여기며 살아간다 그렇게 해 주신 분께 너무 죄송스럽구 미안하구 또 고맙습니다 제발 저와 같이

제목: 6월 민주 항쟁
카테고리: 20대 초반 민주화라는 담대한 목표를 가지고 친구들과 함께 민주화 운동을 하던 시절을 회상하는 내용 담은 수필
필자: 민주운동을 관심을 가지고 6월 민주 항쟁 시위를 하던 대학생 
내용: 민주화운동이 일어난 지 40년, 우리나라의 민주주의가 이만큼 성장한 건 분명 사실이나 여전히 사회 곳곳에서 많은 이들이 부당함과 억울함 속에 살아가야만 한다. 특히 우리나라도 이제 막 경제 발전 단계로 진입하고 있다 보니 과거와 같지 않게 물가 상승률도 빠르며 각종 부동산 가격 등 불투명성이 커지기 시작했는데 이러한 것 또한 현재 대한민국 국민으로서 살아간다는 점 하나 만으로 크다고 느끼기도 어렵기에 더욱 더 체감 상 높아진 현실이라고 생각한다.. 그렇게 나에게 이런 일? 은 상상조차 할 수 없었지만 갑작스럽고도 갑작스레 내게 찾아왔으며 그 사건 덕분인지 내 삶에도 큰 변화점 3가지 1)직장생활 2%)저축습관3>직업 및 진로 선택 4<진로 고민 -취업 준비 과정 5 <